In [7]:
%matplotlib inline

import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from numpy import mean
from math import sqrt,acos,cos,sin,pi,exp,log,isnan,atan2
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from numpy import asarray
from root_pandas import read_root
from matplotlib import gridspec
from scipy import stats


datafolder = '../../data'
dumpfolder = '../PlotDumps'

RSE  = ['run','subrun','event']

In [8]:
def proc_df_mc(df_dlana,df_wgts,df_goodrun):
    
    df_full = df_dlana.join(df_goodrun.set_index('run'),on='run')
    #df_full = df_full.join(df_wgts.set_index(RSE)[['nu_interaction_mode','nu_interaction_type','xsec_corr_weight','spline_weight','nu_interaction_ccnc','nu_pdg']],on=RSE)

    df_full_goodruns = df_full.query('good==1')
    df_full_goodruns_precuts = df_full_goodruns.query(s_precut)
    #df_full_goodruns_precuts.insert(0,'mc_label',df_full_goodruns_precuts.apply(bless_MC_labels,axis=1))
    #df_full_goodruns_precuts.insert(0,'int_label',df_full_goodruns_precuts.apply(bless_int_labels,axis=1))
    
    #df_full_nodupes = df_full_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()
    # keep dupes for now...
    return df_full_goodruns_precuts

def proc_detvar_mc(df_dlana,df_cv,df_goodrun):
    
    df_full = df_dlana.join(df_goodrun.set_index('run'),on='run')
    df_full = df_full.join(df_cv.set_index(RSE),on=RSE,rsuffix='_cv')

    df_full_goodruns = df_full.query('good==1')
    df_full_overlap = df_full_goodruns.query('Enu_1m1p > 0 and Enu_1m1p_cv > 0')
    df_full_goodruns_precuts = df_full_overlap.query(s_precut)
    
    return df_full_goodruns_precuts

In [9]:
# Time to load the good runs list
good_run1_df = pd.read_csv('%s/goodruns_2020.txt'%datafolder)
good_run2_df = pd.read_csv('%s/goodruns_2020_run2.txt'%datafolder)
good_run3_df = pd.read_csv('%s/goodruns_2020_run3.txt'%datafolder)

good_run1_df['good'] = 1
good_run2_df['good'] = 1
good_run3_df['good'] = 1

In [10]:
# Beam quality
beamq_df = read_root('%s/beamdataquality_remix_bnb5e19.root'%datafolder,'bdq')

In [11]:
tag = 'June8_yesdupes'

# All Precuts
orthogonalcut = 'MaxShrFrac < .2'
precuts ='PassSimpleCuts == 1 and ChargeNearTrunk > 0 and FailedBoost != 1 and OpenAng > .5'
s_precut = orthogonalcut + ' and ' + precuts

# MC BNB OVERLAY

##  Run1

In [12]:
s_mc = 'numu_run1'
df_mc = read_root('%s/bnb_overlay/mcc9_v28_wctagger_bnboverlay_stripped.root'%datafolder,'FinalVertexVariables')
df_mc_cvweight = read_root('%s/bnb_overlay/weights_forCV_v40_bnb_nu_run1.root'%datafolder)

df_mc_full = proc_df_mc(df_mc,df_mc_cvweight,good_run1_df)  
df_mc_full = df_bnb_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

NameError: name 'df_bnb_full' is not defined

In [ ]:
s_mc = 'numu_lowe_run1'
df_mc = read_root('%s/bnb_overlay/mcc9_v29e_run1_bnb_nu_overlay_LowE.root'%datafolder,'dlana/FinalVertexVariables')
df_mc_cvweight = read_root('%s/bnb_overlay/weights_forCV_v40_bnb_nu_lowE_run1.root'%datafolder)

df_mc_full = proc_df_mc(df_mc,df_mc_cvweight,good_run1_df)  
df_mc_full = df_bnb_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

## Run3

In [57]:
s_mc = 'numu_run3'
df_mc = read_root('%s/bnb_overlay/mcc9_v29e_dl_run3b_bnb_nu_overlay_nocrtremerge_stripped.root'%datafolder,'FinalVertexVariables')
df_mc_cvweight = read_root('%mc/bnb_overlay/weights_forCV_v40_bnb_nu_run3.root'%datafolder)

df_mc_full = proc_df_mc(df_mc,df_mc_cvweight,good_run3_df)  
df_mc_full = df_bnb_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

In [66]:
s_mc = 'numu_lowe_run3'
df_mc = read_root('%s/bnb_overlay/mcc9_v29e_run3b_bnb_nu_overlay_LowE.root'%datafolder,'FinalVertexVariables')
df_mc_cvweight = read_root('%mc/bnb_overlay/weights_forCV_v40_bnb_nu_lowE_run3.root'%datafolder)

df_mc_full = proc_df_mc(df_mc,df_mc_cvweight,good_run3_df)  
df_mc_full = df_bnb_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

# DIRT

In [12]:
df_dirt = read_root('../data/dirt/FVV-Prime-dirt-Mar3-WC-1M1P.root','FinalVertexVariables')
df_dirt_cvweight = read_root('../data/dirt/weights_forCV_v40_dirt_nu_run1.root')

df_full = df_dirt.join(good_run1_df.set_index('run'),on='run')
df_full = df_full.join(df_dirt_cvweight.set_index(RSE)[['nu_interaction_mode','nu_interaction_type','xsec_corr_weight','spline_weight','nu_interaction_ccnc','nu_pdg']],on=RSE)
df_full_goodruns = df_full.query('good==1')
df_full_goodruns_precuts = df_full_goodruns.query('PassPMTPrecut==1 and PassSimpleCuts==1')

with open('/home/dcianci/Physics/1e1p/1mu1pSelection/bdtweights_1mu1p_WC_apr1.pickle','rb') as handle: cosmicBDT,nubkgBDT = pickle.load(handle)          # Load BDT weights for 1mu1p background differentiation    
myvars = ['Eta','ChargeNearTrunk','PT_1m1p','PhiT_1m1p','AlphaT_1m1p','Sph_1m1p','Q0_1m1p','Q2_1m1p','Q3_1m1p','Lepton_ThetaReco','Lepton_PhiReco','Proton_ThetaReco','Proton_PhiReco','PTRat_1m1p','Lepton_TrackLength','Thetas','Phis','Proton_TrackLength','OpenAng','PzEnu_1m1p']
s_nubdtname = 'BDTscore_1mu1p_nu'
s_cosbdtname = 'BDTscore_1mu1p_cosmic'
df_full_goodruns_precuts.insert(0,s_nubdtname,nubkgBDT.predict(df_full_goodruns_precuts[myvars].values.tolist(),output_margin=True))
df_full_goodruns_precuts.insert(0,s_cosbdtname,cosmicBDT.predict(df_full_goodruns_precuts[myvars].values.tolist(),output_margin=True))

df_nodupes = df_full_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()
df_nodupes.to_parquet('../data/pickles/dirt_run1_nodupes%s.parquet'%tag)

del df_full,df_full_goodruns,df_full_goodruns_precuts

7557 1470


# MC NUE OVERLAY

In [11]:
s_mc = 'nue_run1'
df_mc = read_root('%s/nue_intrinsic_overlay/mcc9_v28_wctagger_nueintrinsics_stripped.root'%datafolder,'FinalVertexVariables')
df_mc_cvweight = read_root('%mc/nue_intrinsic_overlay/weights_forCV_v40_intrinsic_nue_run1.root'%datafolder)

df_mc_full = proc_df_mc(df_mc,df_mc_cvweight,good_run1_df)  

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

NameError: name 'df_bnb' is not defined

In [60]:
s_mc = 'nue_run3'
df_mc = read_root('%s/nue_intrinsic_overlay/mcc9_v29e_run3b_bnb_intrinsic_nue_overlay_nocrtremerge_stripped.root'%datafolder,'FinalVertexVariables')
df_mc_cvweight = read_root('%mc/nue_intrinsic_overlay/weights_forCV_v40_intrinsic_nue_run3.root'%datafolder)

df_mc_full = proc_df_mc(df_mc,df_mc_cvweight,good_run3_df)  

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

# EXT

In [62]:
s_ext = 'ext_run1'
df_ext = read_root('%s/ext/mcc9_v28_wctagger_extbnbFULL_stripped.root'%datafolder)
    
df_ext = df_ext.join(good_run1_df.set_index('run'),on='run')
df_ext_goodruns = df_ext.query("good==1")
df_ext_goodruns_precuts = df_ext_goodruns.query(s_precut)

#df_nodupes = df_ext_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_ext_goodruns_precuts.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_ext,tag)
 
del df_ext,df_ext_goodruns,df_ext_goodruns_precuts

In [64]:
s_ext = 'ext_run3'
df_ext = read_root('%s/ext/mcc9_v29e_dl_run3_G1_extbnb_stripped.root'%datafolder)
    
df_ext = df_ext.join(good_run3_df.set_index('run'),on='run')
df_ext_goodruns = df_ext.query("good==1")
df_ext_goodruns_precuts = df_ext_goodruns.query(s_precut)

#df_nodupes = df_ext_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_ext_goodruns_precuts.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_ext,tag)
 
del df_ext,df_ext_goodruns,df_ext_goodruns_precuts

# Data

In [103]:
s_data = 'data_run1_5e19'
df_data = read_root('%s/bnb/mcc9_v28_wctagger_5e19.root'%datafolder,'dlana/FinalVertexVariables')

df_data = df_data.join(good_run1_df.set_index('run'),on='run')
df_data = df_data.join(beamq_df.set_index(['run','subrun','event']),on=['run','subrun','event'])

df_data_goodruns = df_data.query("good==1 and result==1")
df_data_goodruns_precuts = df_data_goodruns.query(s_precut)
#df_nodupes = df_data_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_data_goodruns_precuts.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag)


del df_data,df_data_goodruns,df_data_goodruns_precuts

0


In [19]:
s_data = 'data_run3_1e19'
df_data = read_root('%s/bnb/mcc9_v28_wctagger_run3_bnb1e19.root'%datafolder,'dlana/FinalVertexVariables')

df_data = df_data.join(good_run3_df.set_index('run'),on='run')

df_data_goodruns = df_data.query("good==1")# and result==1")
df_data_goodruns_precuts = df_data_goodruns.query(s_precut)
#df_nodupes = df_data_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_data_goodruns_precuts.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))


del df_data,df_data_goodruns,df_data_goodruns_precuts

In [7]:
s_data = 'data_run2_filter'
#df_data = read_root('%s/bnb/mcc9_v29e_dl_run3_G1_bnb_dlfilter_1m1p_v1_1_2b_fvv.root'%datafolder,'dlana/FinalVertexVariables')
df_data = read_root('%s/bnb/mcc9_v29e_dl_run2_D2_bnb_dlfilter_1m1p_v1_1_2b_fvv.root'%datafolder,'dlana/FinalVertexVariables')


df_data = df_data.join(good_run2_df.set_index('run'),on='run')

df_data_goodruns = df_data.query("good==1")# and result==1")
df_data_goodruns_precuts = df_data_goodruns.query(s_precut)
#df_nodupes = df_data_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_data_goodruns_precuts.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

del df_data,df_data_goodruns,df_data_goodruns_precuts

In [13]:
s_data = 'data_run3_filter'
df_data = read_root('%s/bnb/mcc9_v29e_dl_run3_G1_bnb_dlfilter_1m1p_v1_1_2b_fvv.root'%datafolder,'dlana/FinalVertexVariables')

df_data = df_data.join(good_run3_df.set_index('run'),on='run')

df_data_goodruns = df_data.query("good==1")# and result==1")
df_data_goodruns_precuts = df_data_goodruns.query(s_precut)

df_data_goodruns_precuts.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

del df_data,df_data_goodruns,df_data_goodruns_precuts

In [6]:
s_data = 'data_run1_filter'
df_data = read_root('%s/bnb/mcc9_v29e_dl_run1_C1_bnb_dlfilter_1m1p_v1_1_2b_fvv.root'%datafolder,'dlana/FinalVertexVariables')


df_data = df_data.join(good_run1_df.set_index('run'),on='run')

df_data_goodruns = df_data.query("good==1")# and result==1")
df_data_goodruns_precuts = df_data_goodruns.query(s_precut)
#df_nodupes = df_data_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_data_goodruns_precuts.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

del df_data,df_data_goodruns,df_data_goodruns_precuts

In [6]:
s_data = 'data_run1_makeup'
df_data = read_root('%s/bnb/mcc9_v29e_dl_run1_C1_bnb_dlfilter_1m1p_v1_1_2b_fvv.root'%datafolder,'dlana/FinalVertexVariables')


df_data = df_data.join(good_run1_df.set_index('run'),on='run')

df_data_goodruns = df_data.query("good==1")# and result==1")
df_data_goodruns_precuts = df_data_goodruns.query(s_precut)
#df_nodupes = df_data_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_data_goodruns_precuts.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

del df_data,df_data_goodruns,df_data_goodruns_precuts

# Detector Systematics (run3; not updated)

In [ ]:
s_mc = 'numu_run3'
df_mc = read_root('%s/bnb_overlay/mcc9_v29e_dl_run3b_bnb_nu_overlay_nocrtremerge_stripped.root'%datafolder,'FinalVertexVariables')
df_mc_cvweight = read_root('%mc/bnb_overlay/weights_forCV_v40_bnb_nu_run3.root'%datafolder)

df_mc_full = proc_df_mc(df_mc,df_mc_cvweight,good_run3_df)  
df_mc_full = df_bnb_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

In [14]:
df_cv = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_overlay_CV.root','FinalVertexVariables')

s_data = 'wiremodX'
df_bnb = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_overlay_wiremodX.root','dlana/FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

s_data = 'recomb2'
df_bnb = read_root('../../data/detsys/mcc9_v40_dl_run3b_bnb_nu_overlay_DetVar_recomb2.root','FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

s_data = 'wiremoddEdx'
df_bnb = read_root('../../data/detsys/mcc9_v40_dl_run3b_bnb_nu_overlay_DetVar_wiremodscaleddedx.root','FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [16]:
s_data = 'wiremodThetaXZ'
df_bnb = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_nu_overlay_DetVar_wiremodThetaXZ.root','dlana/FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [17]:
s_data = 'wiremodThetaYZ'
df_bnb = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_nu_overlay_DetVar_wiremodThetaYZ.root','dlana/FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

s_data = 'LYAttenuation'
df_bnb = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_nu_overlay_DetVar_LYAttenuation.root','dlana/FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

s_data = 'SCE'
df_bnb = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_nu_overlay_DetVar_SCE.root','dlana/FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

s_data = 'LYdown'
df_bnb = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_nu_overlay_DetVar_LYdown.root','dlana/FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

s_data = 'wiremodYZ'
df_bnb = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_nu_overlay_DetVar_wiremodYZ.root','dlana/FinalVertexVariables')
df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

#s_data = 'wiremodX'
#df_cv = read_root('../../data/detsys/mcc9_v40a_dl_run3b_bnb_overlay_CV.root','FinalVertexVariables')
#df_bnb_proc = proc_detvar_mc(df_bnb,df_cv,good_run3_df)
#df_bnb_proc.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [8]:
df_data = read_root('../../data/bnb/mcc9_v29e_dl_run3_G1_bnb_dlfilter_1m1p_v1_1_2b_fvv.root','dlana/FinalVertexVariables')

df_data = df_data.join(good_run3_df.set_index('run'),on='run')
#df_data = df_data.join(beamq_df.set_index(['run','subrun','event']),on=['run','subrun','event'])

df_data_goodruns = df_data.query("good==1")# and result==1")
df_data_goodruns_precuts = df_data_goodruns.query(s_precut)

#df_nodupes.to_parquet('../../data/pickles/data_run1_filtertest_nodupes.parquet')
df_data_goodruns_precuts.to_parquet('../../data/pickles/data_run3_filtertest_yesdupes.parquet')

del df_data,df_data_goodruns,df_data_goodruns_precuts

In [7]:
df_data = read_root('../data/bnb/mcc9_v29e_dl_run3_G1_bnb_dlfilter_1m1p_fvv.root','dlana/FinalVertexVariables')

df_data = df_data.join(good_run3_df.set_index('run'),on='run')
#df_data = df_data.join(beamq_df.set_index(['run','subrun','event']),on=['run','subrun','event'])

df_data_goodruns = df_data.query("good==1")# and result==1")
df_data_goodruns_precuts = df_data_goodruns.query("PassPMTPrecut==1 and PassSimpleCuts==1")
df_nodupes = df_data_goodruns_precuts.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_nodupes.to_parquet('../data/pickles/data_run3_filter_nodupes%s.parquet'%tag)

del df_data,df_data_goodruns,df_data_goodruns_precuts

In [9]:
#FAKEDATA
df_data = read_root('../data/bnb/FVV_fakedata_test.root','dlana/FinalVertexVariables')

df_data = df_data.query("PassPMTPrecut==1 and PassSimpleCuts==1")
df_nodupes = df_data.sort_values('BDTscore_1mu1p_cosmic',ascending=False).drop_duplicates(RSE).sort_index()

df_nodupes.to_parquet('../data/pickles/data_fakedata_nodupes%s.parquet'%tag)

del df_data